In [4]:
# 8 threads
# 14 tests < 1000 fidx takes around 2h
# !TEST_NAME='jsc' FIDX_LIMIT=1000 python rq1.py # 42 mins
# !TEST_NAME='boa' FIDX_LIMIT=1000 python rq1.py # 58 mins
# !TEST_NAME='commanderkeen' FIDX_LIMIT=1000 python rq1.py # 144 mins
# !TEST_NAME='funky-kart' FIDX_LIMIT=1000 python rq1.py # 155 mins
!FIDX_LIMIT=1000 ONLY_FAIL=True python rq1.py

RQ1
RUNNING: wamr#2861 with 1 functions
FINISHED: wamr#2861 in 0.0005226731300354004 minutes
RUNNING: wasmedge#3018 with 12 functions
FINISHED: wasmedge#3018 in 0.0006904959678649903 minutes
RUNNING: wamr#2789 with 58 functions
FINISHED: wamr#2789 in 2.0017606178919474 minutes
RUNNING: wamr#2450 with 66 functions
FINISHED: wamr#2450 in 0.0005558609962463378 minutes
RUNNING: wamr#2862 with 73 functions
FINISHED: wamr#2862 in 0.0005324681599934896 minutes
RUNNING: wasmedge#3019 with 76 functions
FINISHED: wasmedge#3019 in 0.0005364894866943359 minutes
RUNNING: wasmedge#3076 with 131 functions
FINISHED: wasmedge#3076 in 0.0005415439605712891 minutes
RUNNING: mandelbrot with 167 functions
FINISHED: mandelbrot in 0.000533743699391683 minutes
RUNNING: guiicons with 505 functions
FINISHED: guiicons in 0.00053253968556722 minutes
RUNNING: rguilayout with 510 functions
FINISHED: rguilayout in 0.0005095998446146647 minutes
RUNNING: rguistyler with 575 functions
FINISHED: rguistyler in 0.00052140

In [6]:
!python rq2.py

['boa', 'guiicons', 'funky-kart', 'jsc', 'rfxgen', 'rguilayout', 'rguistyler', 'riconpacker', 'sqlgui', 'commanderkeen', 'hydro', 'rtexviewer', 'mandelbrot', 'wasmedge#3057', 'wasmedge#3076', 'wamr#2450', 'wasmedge#3019', 'wamr#2789', 'wamr#2862', 'wasmedge#3018', 'wamr#2861']
len(testset): 21
